In [1]:
%pylab inline
import warnings                               # Suppressing annoying Librosa warnings
warnings.filterwarnings('ignore')

import os
from os import walk
import shutil                                 # For shell commands, accessing and manipulating files and directories

from icecream import ic
import math
import numpy as np
import pandas as pd                           # Data analysis
from collections import Counter
from scipy.stats import pearsonr
from sklearn import metrics as sk

import librosa                                # Manipulating sound files
from librosa import display
import IPython.display as ipd
import soundfile
import glob 


import tensorflow as tf                       # Deep Learning
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, LambdaCallback, TensorBoard
from imblearn.over_sampling import SMOTE


import matplotlib.pyplot as plt               # Visualizations
from tqdm import tqdm

from sklearn.preprocessing import normalize   # Data Engineering
from sklearn.model_selection import train_test_split


import tkinter as tk
from PIL import ImageTk, Image
import cv2

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Path might require modification

model_a = tf.keras.models.load_model('saved_model/model_angle_2.h5')
model_d = tf.keras.models.load_model('saved_model/model_distance_2.h5')
a_dict = {0: 0,
              20: 1,
              40: 2,
              60: 3,
              80: 4,
              100: 5,
              120: 6,
              140: 7,
              160: 8,
              180: 9,
              200: 10,
              220: 11,
              240: 12,
              260: 13,
              280: 14,
              300: 15,
              320: 16,
              340: 17}
d_dict = {10: 0,
              20: 1,
              30: 2,
              40: 3,
              50: 4,
              60: 5,
              70: 6,
              80: 7,
              90: 8,
              100: 9,
              110: 10,
              120: 11,
              130: 12,
              140: 13,
              150: 14,
              160: 15,
              170: 16,
              180: 17,
              190: 18,
              200: 19,
              210: 20,
              220: 21,
              230: 22,
              240: 23,
              250: 24,
              260: 25,
              270: 26,
              280: 27,
              290: 28,
              300: 29}
a_dict=  {val:key for key, val in a_dict.items()}
d_dict=  {val:key for key, val in d_dict.items()}

In [3]:
def visualize_inference(a_inference, d_inference, window=False):
    # Path might require modification
    path = './compass.png'

    img = Image.open(path)
    img = img.resize((300,300), Image.ANTIALIAS)
    img = cv2.cvtColor(numpy.array(img), cv2.COLOR_RGB2BGR)

    start_point = (150, 150) 

    length = 71
    angle = np.radians(a_inference)
    dist = d_inference

    end_point = (int(150 + length * np.sin(angle)), int(150 - length * np.cos(angle)))
 
    color = (255, 0, 0) 

    thickness = 5


    img = cv2.arrowedLine(np.asarray(img), start_point, end_point,
                                         color, thickness) 
    # text
    text = f'{dist} m to {a_inference} degrees'

    # font
    font = cv2.FONT_HERSHEY_SIMPLEX

    # org
    org = (35, 290)

    # fontScale
    fontScale = 0.75

    # Red color in BGR
    color = (255, 0, 0)

    # Line thickness of 2 px
    thickness = 2

    # Using cv2.putText() method
    image = cv2.putText(np.asarray(img), text, org, font, fontScale, 
                     color, thickness, cv2.LINE_AA, False)

    if window:
        img = ImageTk.PhotoImage(Image.fromarray(img), master=window)
    else:
        img = ImageTk.PhotoImage(Image.fromarray(img))

    return img

In [ ]:
window = tk.Tk()
window.title("Join")
window.geometry("300x300")
window.configure(background='grey')

img = Image.open('./compass.png')
img = img.resize((300,300), Image.ANTIALIAS)
img = cv2.cvtColor(numpy.array(img), cv2.COLOR_RGB2BGR)
img = ImageTk.PhotoImage(Image.fromarray(img))

panel = tk.Label(window, image = img)

#The Pack geometry manager packs widgets in rows or columns.
panel.pack(side = "bottom", fill = "both", expand = "yes")

#Start the GUI

window.lift()
window.attributes('-topmost',True)
#window.after_idle(root.attributes,'-topmost',False)

# This path must be set to point to the unity app audio output directory, which should be similar to this
path = 'C:\\Users\\Aaron.Festinger\\AppData\\LocalLow\\Seeing Ear Bats\\Hack-a-thon_2021\\'
#print('Your folder path is "',path,'"')
before = dict ([(f, None) for f in os.listdir (path)])

def dothis():
    global before
    after = dict ([(f, None) for f in os.listdir (path)])
    added = [f for f in after if not f in before]
    if added:
            #print("Added: ", ", ".join (added))
            sample, _ = librosa.load(path+added[-1])
            sample = np.array(sample[1000:16000])
            sample = np.reshape(sample, (1, sample.shape[0], 1))

            before = dict ([(f, None) for f in os.listdir (path)])
            change_image(sample)
    window.after(1000,dothis)

def change_image(obs):
    global window
    img = visualize_inference(a_dict[np.argmax(model_a.predict(obs))], d_dict[np.argmax(model_d.predict(obs))], window=window)
    panel.configure(image=img)
    panel.image = img

    
    
window.after(500, dothis)
window.mainloop()